## LangChain tools
Tools w LangChain to zewnętrzne funkcje lub usługi (np. wyszukiwarka, kalkulator, API), które agent może wywoływać podczas rozwiązywania zadań, aby rozszerzyć swoje możliwości poza samo generowanie tekstu.

### Instalacja bibliotek

In [ ]:
!pip install -q langchain langchain-openai python-dotenv

In [2]:
from dotenv import load_dotenv
load_dotenv()

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
print("Model gotowy.")


Model gotowy.


### Proste narzędzie (kalkulator)

In [12]:
from langchain.tools import tool

@tool
def multiply_numbers(x: int, y: int) -> int:
    """Mnoży dwie liczby całkowite."""
    return x * y

print(multiply_numbers.invoke({"x": 6, "y": 7}))


42


### Użycie narzędzia z LLM

In [3]:
from langchain.agents import initialize_agent, AgentType

tools = [multiply_numbers]

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Model sam zdecyduje, że musi użyć multiply_numbers
agent.run("Ile wynosi dwukrotność liczby ludzi na Ziemi?")


/tmp/ipykernel_10207/1539811230.py:5: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(
/tmp/ipykernel_10207/1539811230.py:13: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Ile wynosi dwukrotność liczby ludzi na Ziemi?")




> Entering new AgentExecutor chain...
Thought: Najpierw muszę ustalić aktualną liczbę ludzi na Ziemi. Szacuje się, że w 2023 roku liczba ta wynosi około 8 miliardów. Dwukrotność tej liczby to 16 miliardów. Jednak, aby być precyzyjnym, mogę obliczyć to jako mnożenie 8 miliardów przez 2. 

Action:
```
{
  "action": "multiply_numbers",
  "action_input": {"x": 8000000000, "y": 2}
}
```

Observation: 16000000000
Thought:Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów."
}
```


> Finished chain.


'Dwukrotność liczby ludzi na Ziemi wynosi 16 miliardów.'

### Kolejne narzędzie (prosta baza wiedzy)

In [13]:
@tool
def company_info(name: str) -> str:
    """Zwraca podstawowe informacje o firmie na podstawie nazwy."""
    companies = {
        "OpenAI": "Firma zajmująca się sztuczną inteligencją, twórca ChatGPT.",
        "LangChain": "Biblioteka ułatwiająca budowanie aplikacji z modelami językowymi.",
        "Tesla": "Producent samochodów elektrycznych i baterii."
    }
    return companies.get(name, "Nie znam tej firmy.")

tools = [company_info]
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("Powiedz mi, czym zajmuje się firma Tesla.")




> Entering new AgentExecutor chain...
Muszę uzyskać podstawowe informacje o firmie Tesla, aby dowiedzieć się, czym się zajmuje.  
Action: company_info  
Action Input: "Tesla"  
Observation: Producent samochodów elektrycznych i baterii.
Thought:I now know the final answer  
Final Answer: Firma Tesla zajmuje się produkcją samochodów elektrycznych i baterii.

> Finished chain.


'Firma Tesla zajmuje się produkcją samochodów elektrycznych i baterii.'

### Dostępne tool w LangChain
Lista dostępnych narzędzi: https://python.langchain.com/docs/integrations/tools/

In [9]:
!pip install wikipedia tavily-python

  Using cached wikipedia-1.4.0.tar.gz (27 kB)
  Preparing metadata (setup.py) ... done
  DEPRECATION: Building 'wikipedia' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'wikipedia'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11757 sha256=d3adbc107029e4347a175b213ffeeaad88fb70f5e2a2849969e93139b4233e22
  Stored in directory: /home/michal/.cache/pip/wheels/79/1d/c8/b64e19423cc5a2a339450ea5d145e7c8eb3d4aa2b150cde33b
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [tavily-python]


In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.agents import initialize_agent, AgentType

# Inicjalizacja narzędzia wyszukiwarki
# klucz API do Tavily: https://app.tavily.com/home
search = TavilySearchResults(max_results=3)

# Bezpośrednie użycie narzędzia
query = "Najnowsze wiadomości o sztucznej inteligencji"
results = search.invoke(query)

print(f"Wyniki wyszukiwania dla zapytania: '{query}'")
for i, result in enumerate(results, 1):
    print(f"\n{i}. {result.get('title', 'Brak tytułu')}")
    print(f"   URL: {result.get('url', 'Brak URL')}")
    print(f"   Treść: {result.get('content', 'Brak treści')[:200]}...")

# Lista narzędzi z wyszukiwarką
tools = [search]

# Agent z dostępem do internetu
agent_with_search = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Przykładowe zapytanie wymagające wyszukiwania w internecie
response = agent_with_search.run(
    "Kto jest obecnym premierem Polski i ile ma lat?"
)
print(f"\nOdpowiedź agenta:\n{response}")

Wyniki wyszukiwania dla zapytania: 'Najnowsze wiadomości o sztucznej inteligencji'

1. Sztuczna inteligencja (AI) - Spider's Web
   URL: https://spidersweb.pl/tag/sztuczna-inteligencja
   Treść: Powiązane tematy: Microsoft, Bing, Copilot, Google, Gemini, Deepseek

AI czyli Artificial Intelligence

Sztuczna Inteligencja - najnowsze wiadomości

[Tech

12.09.2025 10:00

## Albania ma nowego mini...

2. sztuczna Inteligencja - Najnowsze informacje - WP Wiadomości
   URL: https://wiadomosci.wp.pl/tag/sztuczna%20inteligencja
   Treść: Według agencji Bloomberg we wtorek Donald Trump ogłosi wart 70 miliardów dolarów plan inwestycji. Pieniądze mają zostać wydaje na rozwój dwóch obszarów - sztucznej inteligencji i energetyki.

11 lipca...

3. Sztuczna Inteligencja - najnowsze informacje - Strona 1 - Fintek.pl
   URL: https://fintek.pl/aktualnosci/sztuczna-inteligencja/
   Treść: #Sztuczna Inteligencja
 #Technologie

10 czerwca 2025 07:05

Meta szykuje rekordową inwestycję w sztuczną inteligencję

/tmp/ipykernel_25809/2051081279.py:21: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent_with_search = initialize_agent(
/tmp/ipykernel_25809/2051081279.py:29: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = agent_with_search.run(


Muszę znaleźć aktualne informacje na temat obecnego premiera Polski oraz jego wieku. 
Action: tavily_search_results_json
Action Input: "obecny premier Polski 2023"
Observation: [{'title': 'Premierzy Polski – Wikipedia, wolna encyklopedia', 'url': 'https://pl.wikipedia.org/wiki/Premierzy_Polski', 'content': '| (13.) |  | DonaldTusk(PO)(ur. 1957) | 11 grudnia 2023 | 13 grudnia 2023 | 12 grudnia 202311 czerwca 2025 |  |  | 644 dni | Trzeci rząd Donalda Tuska | Andrzej Duda(2015–2025) |\n| Karol Nawrocki(od 2025) |\n| Stan na 17 września 2025 | [...] Morawiecki (2017–2023)  Donald Tusk (od 2023) | [...] | 14 listopada 2019 | 15 listopada 2019 | 19 listopada 20194 czerwca 2020 | 13 listopada 2023 | 27 listopada 2023 | Drugi rząd Mateusza Morawieckiego | Sejm RP IX kadencji | Andrzej Duda(2015–2025) |\n| 13 listopada 2023 | 27 listopada 2023 | 11 grudnia 2023 | 11 grudnia 2023 | 13 grudnia 2023 | Trzeci rząd Mateusza Morawieckiego | Sejm RP X kadencji | Andrzej Duda(2015–2025) |', 'score': 0

In [14]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Inicjalizacja narzędzia Wikipedia
wikipedia = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())

# Bezpośrednie użycie
wiki_result = wikipedia.run("Python programming language")
print(f"Wikipedia - Python:\n{wiki_result[:300]}...")

### Agent z wieloma narzędziami
# Agent z dostępem do wielu źródeł informacji
tools_extended = [search, wikipedia, company_info]

agent_multi_tool = initialize_agent(
    tools=tools_extended,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Zapytanie wymagające użycia różnych narzędzi
response = agent_multi_tool.run(
    "Znajdź informacje o Pythonie w Wikipedii, a następnie wyszukaj "
    "najnowsze artykuły o nowych funkcjach w Python 3.12"
)
print(f"\nOdpowiedź:\n{response}")

Wikipedia - Python:
Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, ...


> Entering new AgentExecutor chain...
Muszę najpierw znaleźć ogólne informacje o Pythonie w Wikipedii, a następnie poszukać najnowszych artykułów dotyczących nowych funkcji w Python 3.12. 

Action: wikipedia  
Action Input: Python (programming language)  
Observation: Page: Python (programming language)
Summary: Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation.
Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.
Guido van Ros